In [1]:
import pandas as pd
import numpy as np
import pdfquery
import json
from shapely.geometry import box
from shapely.ops import cascaded_union
import pdftableextract as pte
from math import floor
import os
from os import listdir
from os.path import isfile, join, isdir
import time
import re
import pickle
import hashlib
import unidecode
from fuzzysearch import find_near_matches
import textract
import unicodedata
from pymongo import MongoClient
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
#from configuration.configuration import ConfigClass,DbConf
import shutil
import datetime
import multiprocessing as mp
import datefinder
import zipfile
import signal
import warnings
warnings.filterwarnings("ignore")

PDF_DIR = ""
temp_fdf = pd.DataFrame()
classi_fdf = pd.DataFrame()
extract_fdf = pd.DataFrame()

#class Document_Analysis:
    
# SELECT max(batch_id) FROM file_classification;
#     def keywordimport():
#         engine = create_engine(ConfigClass.SQLALCHEMY_DATABASE_URI)
        
#         susp_df= pd.read_sql_query('SELECT k.id,k.file_class as fileclass,k.file_type as filetype,\
#                                    k.keyword,k.remove_class FROM suspend_keywords k',engine)
#         keyword_df= pd.read_sql_query('SELECT k.id,k.file_class as fileclass,k.file_type as filetype,k.purpose,\
#                                     k.decision_type,k.keyword,k.bias as bias,k.sub as sub FROM keywords k',engine)
#         keyword_df['sub']=keyword_df['sub'].apply(lambda x : json.loads(x) if x!=None else [])
#         keyword_df['keyword']=keyword_df['keyword'].apply(lambda x : json.loads(x))

#         return keyword_df, susp_df

#Getting number of pages in a pdf file    
#     def get_pgnum(filename):
#         pdf = pdfquery.PDFQuery(ConfigClass.UPLOAD_FOLDER + "/" + filename)
#         pdf.load()
#         pgn = len(pdf.tree.getroot().getchildren())
#         return pgn

 #Takes a dataframe of filegroups which also contains the list of files as its argument and returns a dataframe files with columns, name,file group and its filetype(right now its either ticket or other as the file with smallest name in a filegroup is ticket)
def get_structured_files_dataframe(df):
    tmp = pd.DataFrame(columns=["file","group"])
    j = 0
    for i, r in df.iterrows():
        for f in r.files:
            tmp.loc[j] = [f, r.group]
            j = j+1
    tmp["ext"] = tmp.apply(lambda x : x.file.split(".")[-1],axis =1)
    tmp["length"] = tmp.apply(lambda x : len(x.file),axis =1)
    TI, OI = [], []
    for case, indices in list(tmp.groupby("group").groups.items()):
        ticket_index = tmp.loc[indices].length.values.argmin()
        for i, idx in enumerate(indices):
            if i == ticket_index:
                TI.append(idx)
            else:
                OI.append(idx)
    tmp['type'] = "OTHER"
    tmp.loc[TI,"type"] = "TICKET"
    return tmp

    #takes a df and returns to json, the df is taken from the data extracted from the tables of pdfs using pdf tables extract
def df_to_json(df):
    js = {}
    vals = df[0].values
    ls = []
    for i, v in enumerate(vals):
        if not v:
            ls.append(ls[-1])
        else:
            ls.append(v)
    df[0] = ls
    for key, idx in list(df.groupby([0]).groups.items()):
        js[key] = {}
        for val in df.loc[idx][[1,2]].values:
            if val[0] == "" and val[1] == "":
                js[key]["_value"] = ""
            elif val[0] == "" and val[1] != "":
                js[key]["_value"] = val[1]
            elif val[0] != "" and val[1] == "":
                js[key]["_value"] = val[0]
            elif val[0] != "" and val[1] != "":
                js[key][val[0]] = val[1]                
        if(key=="Documentos"):
            js[key] = df.loc[idx][[1,2]].values.tolist()
        if len(js[key]) == 1 and "_value" in js[key]:
            js[key] = js[key]["_value"]
    return js

    #takes the path of the pdf and returns its text using pdf query the text extracted will be sorted accoring to its y cooridinates of its bounding boxes
def get_pdf_text(path):
    try:
        pdf=pdfquery.PDFQuery(path)
        pdf.load()
        pdftext=""
        pgn=len(pdf.tree.getroot().getchildren())
        for i in range(0,pgn):
            root = pdf.tree.getroot().getchildren()[i]
            npg=[]
            for node in root.iter():
                try:
        #             if node.tag == "LTTextLineHorizontal" or node.tag == "LTTextBox" or node.tag=="LTTextBoxHorizontal":
                    if node.text and float(node.get("y1"))>50 and float(node.get("x1"))>50:
                        npg.append(node)
        #                     pdftext=pdftext+"\n"
                except Exception as e:
                    print((node.tag, e))
            npg=sorted(npg, key=lambda x: float(x.get("x0")))
            npg=sorted(npg, key=lambda x: round(float(x.get("y0"))),reverse=True)
            if len(npg)>0:
                prev_y=round(float(npg[0].get("y0")))
                for x in npg: 
                    esc="\n"
                    if round(float(x.get("y0")))==prev_y:
                        esc="|"
    #                         print(x.get("y0"),x.get("y1"),x.text)
                    prev_y=round(float(x.get("y0")))
                    if len(x.text)> 0:
                        pdftext+=esc+x.text
        return pdftext
    except Exception as e:
        return('Error:'+str(e))

 #takes rtf files path and returns its text    
def get_rtf_text(path):
    text = os.popen('unrtf --text '+path).read()
    return text

    #takes the path of a pdf and extract table 1 and table 2 of tickets and returns its json 
def parse_ticket(pdf_path):
    _JSON = {"filepath" : pdf_path}
    try:
        pdf = pdfquery.PDFQuery(pdf_path)
        pdf.load()
        root = pdf.tree.getroot().getchildren()[0]
        page_box = [float(x) for x in root.get("bbox")[1:-1].split(",")]
        tables, _ =\
        list(zip(
            *sorted(
                [(p.bounds,p.area) for p in cascaded_union(
                    [box(*[float(x) for x in node.get("bbox")[1:-1].split(",")]) for node in root.iter() if node.tag == "LTRect"]
                )],
                key = lambda x : -x[1]
            )
        ))
        X = page_box[2]
        Y = page_box[3]
        xf = 11.69/X
        yf = 8.27/Y
        t1, t2 = tables

        table_1_bbox = ":".join(map(str,(t1[0]*xf - 0.1, (Y - t1[3])*yf - 0.1, t1[2]*xf + 0.1, (Y - t1[1])*yf + 0.1)))
        table_2_bbox = ":".join(map(str,(t2[0]*xf - 0.1, (Y - t2[3])*yf - 0.1, t2[2]*xf + 0.1, (Y - t2[1])*yf + 0.1)))

        df1 =\
        pd.DataFrame(
            pte.table_to_list(
                pte.process_page(
                    pdf_path,
                    "1",
                    crop = table_1_bbox,
                    pad=20
                ),
                "1"
            )[1]
        )
        _JSON["table_1"] = df_to_json(df1)
        df2 = \
        pd.DataFrame(
            pte.table_to_list(
                pte.process_page(
                    pdf_path,
                    "1",
                    crop = table_2_bbox,
                    pad=20
                ),
                "1"
            )[1]
        )
        df2.columns = df2.iloc[0]
        df2 = df2.reindex(df2.index.drop(0))
        _JSON["table_2"] = df2.to_json(orient='index')
    except Exception as e:
            return('Error:'+str(e))
    return json.dumps(_JSON, ensure_ascii=False)

    #takes a path of a file(pdf and rtf) and extract its texts and remove its accents of spansish characters
def parse_other(pf):
    text=""
    try:
        if pf[-3:].lower()=='rtf':
            #print('rtf')
            newtext = unidecode.unidecode(textract.process(pf).decode('utf-8'))
            #newtext =textract.process(pf)
            #newtext=str(newtext,'utf-8')
            # newtext=newtext
        else:
            newtext= get_pdf_text(pf)
        if(len(newtext.split())==0):
            #print("scan")
            newtext =textract.process(pf,method='tesseract').decode('utf-8')
        try:
            newtext=(''.join((c for c in unicodedata.normalize('NFD', newtext) if unicodedata.category(c) != 'Mn'))).lower()
        except Exception as e:
            print((str(e)+" binary ",pf[-3:].lower()))
            newtext=(''.join((c for c in unicodedata.normalize('NFD', newtext.decode("utf-8")) if unicodedata.category(c) != 'Mn'))).lower()
        rem=''
        paratlist=['MODO DE IMPUGNACION:'.lower(),'mode d\'impugnacio',
                   'recurso de repelacion','recurs de reposicio','recurso de reposicion','recurso de apelacion',
                    'INTERPONER RECURSO DIRECTO DE REVISION']

        for parat in paratlist:
            if (parat.lower() in newtext) :
                rem=newtext.split(parat.lower())[-1]
        newtext=newtext.replace(rem,'')
    except Exception as e:
        newtext='Error:'+str(e)
    return newtext

def unzip_add(fdf,PDF_DIR):
    ffdf = fdf.copy()
    #import zipfile
    try:
        for i, r in fdf.iterrows():
            zip_dict ={}
            if r.filename[-3:].lower()=='zip':
                z_files = []
                #print((r.filename.split('.')[0]))
                with zipfile.ZipFile(join(PDF_DIR,r.filename)) as z:
                    for fileinzip in [x for x in z.namelist()]:
                        if not os.path.isdir(fileinzip):
                            try:
                                #fileinzip = unidecode.unidecode(fileinzip)
                                zfdir=join(PDF_DIR, os.path.basename(fileinzip))
                            except Exception as e:
                                fileinzip = unidecode.unidecode(fileinzip)
                                zfdir=join(PDF_DIR, os.path.basename(fileinzip))
                            try:
                                with z.open(os.path.join(fileinzip)) as fz,open(zfdir, 'wb') as zfp:
                                    shutil.copyfileobj(fz, zfp)
                                    #os.remove(zfdir)
                                    if fileinzip.count('.') > 1:
                                        condition = '.' in fileinzip
                                        while(condition):
                                            ind = fileinzip.find('.')
                                            if ind != fileinzip.rfind('.'):
                                                fileinzip = fileinzip[0:ind] + '_' + fileinzip[ind+1:]
                                            else:
                                                condition = False

                                    f_name = r.filename[:-4]+'_'+str(''.join(fileinzip.split()))
                                    f_name = f_name.replace('/','_')
                                    if r.filename[:-4] in f_name:
                                        if not os.path.exists(join(PDF_DIR,f_name)) and not os.path.exists(join(PDF_DIR,unidecode.unidecode(f_name))):
                                            shutil.move(zfdir, join(PDF_DIR,unidecode.unidecode(f_name)))
                                            z_files.append({"filename":unidecode.unidecode(f_name),"filegroup":r['filegroup'],\
                                                "filetype":"OTHER",'ext':fileinzip[-3:],"length":len(fileinzip)})
                                        else:
                                            os.remove(zfdir)
                            except Exception as e:
                                 print(e)
                zip_dict[r['filegroup']]= z_files                                                                  
            if zip_dict !={}:
                for k,val in list(zip_dict.items()):
                    for v in val:
                        ffdf = ffdf.append(pd.Series(v), ignore_index=True) 
    except Exception as e:
        print(str(e))           
    return ffdf         
       

#takes file data frame and returns its table response(table json) and text response
def parsefile(file_name):
    global PDF_DIR
    global temp_fdf
    def timeout(signum, frame):
        raise ValueError('time out,a very specific bad thing happened.')
    signal.signal(signal.SIGALRM, timeout)
    df = temp_fdf.loc[temp_fdf['filename']==file_name]
    zips_filegroup = {}
    for i, r in df.iterrows():
        ticresponse=""
        textresponse=""
        zip_dict = []
        try:
            signal.alarm(45)
            while 1:
                ticresponse= parse_ticket(join(PDF_DIR,r.filename))
                if len(ticresponse)>5:
                    signal.alarm(0)
                    break
        except Exception as e:
            ticresponse='Error:'+str(e)
        try:
            signal.alarm(45)
            if r.filename[-3:].lower()!='zip':
                while 1:
                    if r.filetype=='TICKET':
                        textresponse=ticresponse
                    else:
                        textresponse= parse_other(join(PDF_DIR,r.filename))
                        if 'Error:' in textresponse:
                            print(("text",i,textresponse))
                    if len(ticresponse)>5:
                        signal.alarm(0)
                        break
        except Exception as e:
            print(e)
            textresponse='Error:'+str(e)
            if (r.filename[-3:].lower()=='rtf'):
                print("___________________________________________________")

        df.loc[i,"table_response"] = ticresponse
        df.loc[i,"text_response"] = textresponse
    return df

    # Once the table json is extracted we know which is the principal notification file and moves on to update it
def update_filetype(fdf):
    fgs=fdf.groupby('filegroup')
    fgdf=pd.DataFrame(columns=['filegroup'])
    i=0
    fgdf['files']=np.empty((len(fgs.groups), 0)).tolist()
    fgdf['filetypes']=np.empty((len(fgs.groups), 0)).tolist()
    for k,v in list(fgs.groups.items()):
        fgdf.loc[i,'filegroup']=k
        files=[]
        pcs=[]
        filetypes=[]
        for ind in v:
                files.append(fdf.loc[ind,'filename'])
                filetypes.append(fdf.loc[ind,'filetype'])
        fgdf.loc[i,'filetypes']=filetypes
        fgdf.loc[i,'files']=files
        i+=1
    for i ,r in fdf[~(fdf['table_response'].str.contains('Error:'))&(fdf['filetype']=='TICKET')].iterrows():
        if r['table_response'][:5]!='Error':
            js=json.loads(r['table_response'])
            pf=r['filename'].split('.')[0]+'_'+''.join(js['table_1']['Documentos'][0][0].split()).split('(Principal)')[0]
            fgf=[''.join(x.split())for x in fgdf.loc[fgdf['filegroup']==r['filegroup'],'files'].values[0] ]
            if pf in fgf:
                fl=fgdf.loc[fgdf['filegroup']==r['filegroup'],'files'].values[0][fgf.index(pf)]
                if "caratula" in pf.lower():
                    fdf.loc[fdf['filename'].str.contains(fl),'filetype']="CARATULA"
                else:
                    fdf.loc[fdf['filename'].str.contains(fl),'filetype']="NOTIFICATION"

    return fdf,fgdf 

def read_pdf_n_insert(root_new):
    global PDF_DIR
    global temp_fdf
    global classi_fdf
    global extract_fdf
    file_nm_error = list()

    t_time = time.time()
    PDF_DIR = root_new
    pdf_files= [f for f in listdir(PDF_DIR) if isfile(join(PDF_DIR,  f))]
    if len(pdf_files)>0:
        ls=list()
        for pdf_file in pdf_files:
            if len(pdf_file.split('_'))>=4 and len(pdf_file.split('_')[2])<=4:
                ls.append(pdf_file.split('_'))
            else:
                file_nm_error.append(pdf_file)

        df=pd.DataFrame(ls)
        df = df[pd.notnull(df[3])]
        fg=list(df.groupby(3))
        ls=[]
        i=0
        for k,gdf in  fg:
            fglist=[]
            elist=[]
            flist=list()
            for i, row in gdf.iterrows():
                row=row.dropna()
                fln='_'.join(list(row))
                flist.append(fln)
                elist.append(fln[-3:].lower())
            fgroup={'group':k,'files':flist,'length':len(flist),'min_filename':min(flist, key=len),'extensions':elist}
            ls.append(fgroup)
        flgdf=pd.DataFrame(ls)
        flgdf=flgdf.dropna(thresh=1,axis=1)
        cmpt_fdf = get_structured_files_dataframe(flgdf)
        cmpt_fdf= cmpt_fdf.rename(columns={"file":"filename","group":"filegroup","type":"filetype"})
        cmpt_fdf = unzip_add(cmpt_fdf,PDF_DIR)
        temp_fdf = cmpt_fdf
        list_of_files = cmpt_fdf['filename'].values.tolist()
        print("Parsing start")
        parsing_start_t = time.time()
        pool = mp.Pool(processes=mp.cpu_count())
        results = pool.map_async(parsefile,list_of_files)
        pool.close()
        pool.join()
        print("Parsing complete(time in minutes) = ",(float(time.time()-parsing_start_t)/60))
        output = results.get()
        final_fdf=pd.concat(output)
        fdf,fgdf=update_filetype(final_fdf)
    return fdf


In [2]:
fdf = read_pdf_n_insert("/home/racmo/upload_N27/")

[Errno 13] Permission denied: '/home/racmo/upload_N27/174762389_Prov.  9.10.18 requerimiento a la A.C..RTF'
[Errno 13] Permission denied: '/home/racmo/upload_N27/174520058_Prov. apercibimiento Ac presente informes trimestrales..RTF'
[Errno 13] Permission denied: '/home/racmo/upload_N27/175756928_Prov. traslado alegaciones administrador concursal.RTF'
Parsing start
('text', 30, 'Error:time out,a very specific bad thing happened.')
('text', 39, 'Error:time out,a very specific bad thing happened.')
('text', 109, 'Error:time out,a very specific bad thing happened.')
Parsing complete(time in minutes) =  4.292584685484568


In [3]:
fdf.to_excel("/home/thrymr/552-python-Workspace/shashank_classfication/upload_N27_parse_30_10.xlsx")
#dc.read_pdf_n_insert()
fdf.shape

(135, 7)

### Testing


In [ ]:
fdf2 = read_pdf_n_insert("/home/racmo/upload_N19/")
fdf2.to_excel("/home/thrymr/552-python-Workspace/shashank_classfication/upload_N19_parse_25_10.xlsx")

In [ ]:
newtext =textract.process('/home/racmo/upload_N17/2015_0005652_FIC_20181023080359020181001083011_030_16024_20180928_1000_0017367318_02.rtf')
newtext = ' '.join(newtext.split()).replace('\n', '')

In [4]:
fdf = pd.read_excel("/home/thrymr/552-python-Workspace/shashank_classfication/upload_N27_parse_30_10.xlsx")

In [ ]:
N17_key = [unidecode.unidecode(x).lower() for x in ['DESE CUENTA A S.Sa','SE DA CUENTA A S.Sa','DAR CUENTA A S.Sa','ACUERDO DAR CUENTA A S.Sa','DESE CUENTA A LA MAGISTRADO JUEZ']]
N17_key

In [13]:
N24_key = ['acuerdo el archivo definitivo de las actuaciones','archivar el presente procedimiento','archivar las actuaciones',
 'archivase el expediente','archivense los presentes autos','dando publicidad a la conclusion y archivo','dar por terminado el presente expediente',
 'dar por terminado el presente procedimiento','declarar terminado el presente procedimiento de ejecucion','declaro finalizado el presente procedimiento monitorio',
 'se acuerda dar por terminado el proceso monitorio','se acuerda el archivo de las actuaciones','se acuerda el archivo definitivo',
 'se declara finalizado el presente proceso monitorio','se declara terminado','se decreta la terminacion del proceso','tener por terminado']
N34_key = ['SE ACUERDA EL SOBRESEIMIENTO']
N19_key = ['declarar embargados, por via de mejora de embargo','se acuerda la mejora del embargo','acuerdo la mejora del embargo',
           "disposo la millora de l'embargament",'se declara embargado, por via de mejora','se decreta el embargo, por via de mejora',
           'se declaran embargados, por via de mejora de embargo','se decrete la mejora de embargo',
           'declarar embargada, por via de mejora de embargo','procedase a instancia']
N36_key = ['se alza la suspension de las actuaciones']
N27_key = [x for x in list(kdf.loc[(kdf.fileclass == 'N27')]['keyword'].values)]

In [ ]:
N18_key = ['se decalaran embargados','parte proporcional de la pension que perciba','el embargo de los bienes propiedad del ejecutado',
 'el embargo telematico de las cuentas','el embargo telematico de las devoluciones','declarar embargados los siguientes bienes',
 'para la efectividad de los embargos, materialicense telematicamente desde este juzgado','declaro embargats','per assegurar la trava dels bens embargats',
 "l'ordre d'embargament s'ha de cursar",'la parte proporcional del salario que percibe','la parte proporcional de los salarios',
 'devoluciones que los ejecutados tengan pendientes','para asegurar la traba de los bienes embargados','parte legal del salario que el ejecutado',
 'se declara embargado','el embargo de los bienes','la practica de dichos embargos se debera realizar']

In [21]:
def fussy_match(k):
    f= True
    for kk in k:
        match=find_near_matches(unidecode.unidecode(kk).lower(), text, max_l_dist=1, max_insertions=2)
        if len(match) <= 0:
            f= False
            #print('\\n',match,len(match))
    return f

In [33]:
f ="2012_0000026_S5C_20181023464847220181019135535_021_060154700000000159712018060154700111.PDF"
for i,r in fdf.loc[fdf['filename']==f].iterrows():
    text = r['text_response'].lower()
    text = text.replace('|','')
    #text = ' '.join(text.split())
    print(text)
    for k in N27_key:
        for kk in k:
            if kk.lower() in text:
                print(kk,"--- Present in Text ---")
        f= fussy_match(k)
        if f:
            print(k)

 
jdo. de lo mercantil n. 1 de badajoz 
 -  
c/ castillo puebla de alcocer, 20 
telefono: 924286421, fax: 924286455 
equipo/usuario: irm 
modelo: s40040 
 n .i.g.: 06015 47 1 2012 0000034 
s5c seccion v convenio 0000026 /2012 
procedimiento origen: cno concurso ordinario 0000026 /2012 
sobre otras materias 
 demandante d/na. sistemas de automatismo y control sa 
procurador/a sr/a. jose luis riesco martinez 
abogado/a sr/a. 
 d/na. 
procurador/a sr/a. 
demandantesistemas de automatismo y control sa d/na. abogado/a sr/a. 
 d iligencia de ordenacion 
 s ra. letrado de la administracion de justicia 
dna. lucia peces gomez 
 e n badajoz, a diecinueve de octubre de dos mil dieciocho. 
 p resentado el anterior escrito por el procuradora sr. 
fernandez de arevalo, en nombre y representacion de francisco 
garcia rangel, comunicando el impago de la concursada unase a 
los autos de su razon para constancia en los mismos, dese 
traslado a la concursada por plazo de diez dias, para que 
alegue lo q

In [5]:
with open('Keyword.pickle', 'rb') as handle:
    b=pickle.load(handle, encoding='latin1')
    
kdf=b['keywords']
suspkdf=b['susKeyword']

In [ ]:
kdf.shape

In [16]:
len([x for x in list(kdf.loc[(kdf.fileclass == 'N27')]['keyword'].values)])

1431

In [ ]:
{'N9': [['imposibilidad', 'designar', 'domicilio']], 
 'N16': [['adjudicar ', 'al ejecutante', 'el bien'], ['adjudicar ', 'a la parte ejecutante', 'el bien'], ['adjudicar ', 'a la parte demandante', 'el bien'], ['adjudicar ', 'al ejecutante', 'los bienes'], ['adjudicar ', 'a la parte ejecutante', 'los bienes'], ['adjudicar ', 'a la parte demandante', 'los bienes'], ['consulta a las bases de datos de la aeat', 'telematico', 'bienes'], ['averiguacion', 'telematico', 'bienes']],
 'N19': [['se decreta el embargo, por via de mejora'], ['se decreta el embargo, por via de mejora'], ['procedase a instancia', 'por via de mejora, sobre los bienes']],
 'N10': [['oigase', 'las partes'], ['concede a la parte', 'para que confirme', 'las cantidades'], ['falta de', 'para que confirme', 'las cantidades'], ['faltan', 'para que confirme', 'las cantidades'], ['habiendose omitido', 'para que confirme', 'las cantidades'], ['requerir', 'para que confirme', 'las cantidades'], ['requeriu', 'para que confirme', 'las cantidades'], ['requierase', 'para que confirme', 'las cantidades'], ['requierasele', 'para que confirme', 'las cantidades'], ['requiero', 'para que confirme', 'las cantidades'], ['se le requiere', 'para que confirme', 'las cantidades'], ['se requiere', 'aporte', 'para que confirme', 'las cantidades'], ['se requiera', 'aporte', 'para que confirme', 'las cantidades'], ['una vez acrediten', 'para que confirme', 'las cantidades'], ['hagase saber a la parte que debe aportar', 'para que confirme', 'las cantidades'], ['plazo subsanar defecto', 'para que confirme', 'las cantidades'], ['requeriu-la', 'a fi de que', 'para que confirme', 'las cantidades'], ['para que en el plazo', 'para que confirme', 'las cantidades'], ['se requiera', 'a fin de que facilita el cif correcto', 'para que confirme', 'las cantidades']],
 'N2': [['garcia abascal'], ['acordando que ocupe', 'la posicion ', 'ejecutante'], ['garcia abascal'], ['disposo que ocupi', 'posicio', 'executant'], ['garcia abascal'], ['queda subrogada en la posicion', 'ejecutante']],
 'NX-N9': [['si cambiasen su domicilio']], 
 'N11': [['transferencia a la cuenta bancaria', 'la cantidad'], ['transfierase', 'la cantidad'], ['hagase entrega', 'la cantidad']]}